20170702 실험

더 많은 Places 데이터셋에 대해서 학습한 DCGAN모델을 이용하여 피쳐를 만들고 다시 해본다..

In [1]:
import os
import scipy.misc
import numpy as np
from sklearn.decomposition import PCA

from model import DCGAN
from utils import pp, visualize, to_json, show_all_variables

import tensorflow as tf

from glob import glob

import sys

flags = tf.app.flags
flags.DEFINE_integer("epoch", 25, "Epoch to train [25]")
flags.DEFINE_float("learning_rate", 0.0002, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("train_size", np.inf, "The size of train images [np.inf]")
flags.DEFINE_integer("batch_size", 64, "The size of batch images [64]")
flags.DEFINE_integer("input_height", 64, "The size of image to use (will be center cropped). [108]")
flags.DEFINE_integer("input_width", None,
                     "The size of image to use (will be center cropped). If None, same value as input_height [None]")
flags.DEFINE_integer("output_height", 64, "The size of the output images to produce [64]")
flags.DEFINE_integer("output_width", None,
                     "The size of the output images to produce. If None, same value as output_height [None]")
flags.DEFINE_string("dataset", "PatchofPlaces", "The name of dataset [celebA, mnist, lsun]")
flags.DEFINE_string("input_fname_pattern", "*/*.jpg", "Glob pattern of filename of input images [*]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Directory name to save the image samples [samples]")
flags.DEFINE_boolean("train", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("visualize", False, "True for visualizing, False for nothing [False]")
FLAGS = flags.FLAGS



pp.pprint(flags.FLAGS.__flags)

if FLAGS.input_width is None:
    FLAGS.input_width = FLAGS.input_height
if FLAGS.output_width is None:
    FLAGS.output_width = FLAGS.output_height

if not os.path.exists(FLAGS.checkpoint_dir):
    os.makedirs(FLAGS.checkpoint_dir)
if not os.path.exists(FLAGS.sample_dir):
    os.makedirs(FLAGS.sample_dir)

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth = True


sess = tf.Session(config=run_config)

dcgan = DCGAN(
    sess,
    input_width=FLAGS.input_width,
    input_height=FLAGS.input_height,
    output_width=FLAGS.output_width,
    output_height=FLAGS.output_height,
    batch_size=FLAGS.batch_size,
    sample_num=FLAGS.batch_size,
    dataset_name=FLAGS.dataset,
    input_fname_pattern=FLAGS.input_fname_pattern,
    crop=FLAGS.crop,
    checkpoint_dir=FLAGS.checkpoint_dir,
    sample_dir=FLAGS.sample_dir)

show_all_variables()

if not dcgan.load(FLAGS.checkpoint_dir)[0]:
    raise Exception("[!] Train a model first, then run test mode")

{}
---------
Variables: name (type shape) [size]
---------
generator/g_h0_lin/Matrix:0 (float32_ref 100x8192) [819200, bytes: 3276800]
generator/g_h0_lin/bias:0 (float32_ref 8192) [8192, bytes: 32768]
generator/g_bn0/beta:0 (float32_ref 512) [512, bytes: 2048]
generator/g_bn0/gamma:0 (float32_ref 512) [512, bytes: 2048]
generator/g_h1/w:0 (float32_ref 5x5x256x512) [3276800, bytes: 13107200]
generator/g_h1/biases:0 (float32_ref 256) [256, bytes: 1024]
generator/g_bn1/beta:0 (float32_ref 256) [256, bytes: 1024]
generator/g_bn1/gamma:0 (float32_ref 256) [256, bytes: 1024]
generator/g_h2/w:0 (float32_ref 5x5x128x256) [819200, bytes: 3276800]
generator/g_h2/biases:0 (float32_ref 128) [128, bytes: 512]
generator/g_bn2/beta:0 (float32_ref 128) [128, bytes: 512]
generator/g_bn2/gamma:0 (float32_ref 128) [128, bytes: 512]
generator/g_h3/w:0 (float32_ref 5x5x64x128) [204800, bytes: 819200]
generator/g_h3/biases:0 (float32_ref 64) [64, bytes: 256]
generator/g_bn3/beta:0 (float32_ref 64) [64, byte

## Utility functions

In [2]:
def layer_extraction(dcgan, file_names):
    return dcgan.get_feature(FLAGS, file_names)
    
def maxpooling(disc):
    kernel_stride_size = 4
    maxpooling = [
        tf.nn.max_pool(disc[i],ksize=[1,2**(4-i),2**(4-i),1],
                       strides=[1,2**(4-i),2**(4-i),1],padding='SAME')
        for i in range(4)
    ]

#     tf.global_variables_initializer().run()
    maxpool_result = sess.run(maxpooling)

#     for idx in range(4):
#         print(idx, maxpool_result[idx].shape)

    return maxpool_result

def flatten(disc):
    flatten = [
        tf.reshape(disc[i],[64, -1])
        for i in range(4)
    ]
#     tf.global_variables_initializer().run()
    flatten_result = sess.run(flatten)
    
    return flatten_result

def concat(disc):
    concat = tf.concat(disc,1)

#     tf.global_variables_initializer().run()
    concat_result = sess.run(concat)
    
    return concat_result

def feature_ext_GAN(file_names):
    
    ret = layer_extraction(dcgan, file_names)
    ret = maxpooling(ret)
    ret = flatten(ret)
    ret = concat(ret)
        
    return ret


# Integration

In [3]:
pca = PCA(n_components = 128)

for term in [24]:
    print('%d ~ %d' % (100*term,100*(term+1)))
    
    disc_list = []
    batch_list = []
    file_names = []
    
    for idx in range(100*term,100*(term+1)):
        patch_path ="/media/dongwonshin/Ubuntu Data/Datasets/FAB-MAP/Image Data/City Centre/patches/#300/"
        data = sorted(glob("%s/%04d/*.jpg" % (patch_path, idx)))
#         patch_path ="/media/dongwonshin/Ubuntu Data/Datasets/Places365/Large_images/val_large/patches"
#         data = glob("%s/Places365_val_%08d/*.jpg" % (patch_path, idx))
        file_names.append(data)

    file_names=np.concatenate(file_names)
    print('total:',len(file_names))
#     print(file_names)

    for idx in range(0, len(file_names)-64,64):
        batch_files = file_names[idx: idx+64]

        disc = feature_ext_GAN(batch_files)

        disc_list.append(disc)
        batch_list.append(batch_files)
        sys.stdout.write('.')

    final_disc_list = np.concatenate(disc_list)
    final_batch_list = np.concatenate(batch_list)
    
    X = np.array(final_disc_list)
    pca.fit(X)
    final_disc_list = pca.transform(X)
    
    for idx, name in enumerate(final_batch_list):
        output_filename = '/media/dongwonshin/Ubuntu Data/Datasets/FAB-MAP/Image Data/City Centre/descs/20170702/' + (name.split('/')[-2])+'.desc'
#         output_filename = '/media/dongwonshin/Ubuntu Data/Datasets/Places365/Large_images/val_large/descs/20170702/' + (name.split('/')[-2])+'.desc'
        

        with open(output_filename,'at') as fp:
            for v in final_disc_list[idx]:
                fp.write('%f ' % v)
            fp.write('\n')

print('done.')

2400 ~ 2500
total: 22435
..............................................................................................................................................................................................................................................................................................................................................................done.
